# Python 與機器學習（6）

## 郭耀仁

## 機器學習的技巧

- 機器學習的工作流（Workflow of Machine Learning）
- K-fold 交叉驗證（K-fold Cross Validation）
- 整體學習（Ensemble Learning）
    - 多數決（Majority Voting）
    - 隨機森林（Random Forest）

## 機器學習的工作流

- 以我們一直練習的 **iris** 資料為例：
    - 標準化
    - 應用分類器
- 這些動作會一直在重複應用在訓練與測試資料上
- **Scikit-learn** 提供了一個叫做 **Pipeline** 的工具整理機器學習的工作流

In [1]:
# 機器學習的工作流（2）
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

iris = datasets.load_iris()
X = iris.data # 全部的特徵
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('clf', LogisticRegression(random_state = 0))])

pipe_lr.fit(X_train, y_train)
print('Accuracy: %.2f' % pipe_lr.score(X_test, y_test))

Accuracy: 0.82


## K-fold 交叉驗證

- `K = 10` 的圖示：

![k-fold-cross-validation](https://storage.googleapis.com/py_ml_images/k-fold-cv.png)

## K-fold 交叉驗證（2）

- 不用自己寫
- 透過 `from sklearn.model_selection import cross_val_score` 獲得

In [2]:
# K-fold 交叉驗證（3）
import numpy as np
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator = pipe_lr,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         scoring = 'accuracy')
print(scores)
print("Cross validation accuracy: %.3f" % np.mean(scores))

[ 0.83333333  0.75        0.81818182  0.90909091  1.          0.9         1.
  0.9         1.          0.88888889]
Cross validation accuracy: 0.900


## 整體學習

- 將數個分類器的預測結果綜合考慮
- 以一題是非題來說，假如我們使用一個銅板來決定答案要填是還是非，答對的機率是 50%
- 使用兩個銅板來決定答案，答對的機率是 $1-(50\%)^2=75\%$，如果銅板的數目來到 5 枚，答對的機率是 $1-(50\%)^5=96.875\%$

## 整體學習（2）

- 多數決（Majority Voting）

例如：

分類器 1 -> class 1

分類器 2 -> class 1

分類器 3 -> class 2

分類標籤：**class 1**

In [3]:
# 整體學習（3）
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('clf', LogisticRegression(random_state = 0))])
pipe_ppn = Pipeline([('scl', StandardScaler()),
                    ('clf', Perceptron(eta0 = 0.01, n_iter = 40, random_state = 0))])
pipe_svm = Pipeline([('scl', StandardScaler()),
                    ('clf', SVC(kernel = 'rbf', random_state = 0, gamma = 2))])

eclf = VotingClassifier(estimators = [('lr', pipe_lr), ('ppn', pipe_ppn), ('svm', pipe_svm)], voting = 'hard')
for clf, label in zip([pipe_lr, pipe_ppn, pipe_svm, eclf], ['Logistic Regression', 'Perceptron', 'Support Vector Machine', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv = 10, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.90 (+/- 0.08) [Logistic Regression]
Accuracy: 0.84 (+/- 0.12) [Perceptron]
Accuracy: 0.93 (+/- 0.07) [Support Vector Machine]
Accuracy: 0.92 (+/- 0.07) [Ensemble]


## 整體學習（4）

- 隨機森林（Random Forest）
    - 用 **Bootstrapping** 的方式選出 n 個觀測值（Sampling with replacement）
    - 隨機選 m 個變數
    - 建出多個決策樹
    - 讓多個決策樹進行多數決

In [4]:
# 整體學習（5）
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion = "entropy",
                                n_estimators = 100,
                                random_state = 0)
pipe_rf = Pipeline([('scl', StandardScaler()),
                    ('clf', forest)])

pipe_rf.fit(X_train, y_train)
print('Accuracy: %.2f' % pipe_rf.score(X_test, y_test))

Accuracy: 0.98


## 期末作業

- 參加一個 [Kaggle](https://www.kaggle.com) 競賽：[Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)
- 將 .ipynb 與上傳結果寄至 <tony@kyosei.ai> 這個信箱（準確率大於 50%）
- 主旨請註明：[280_py\_4\_ds\_您的姓名]期末作業
- 繳交期限：2017.04.17 12:00

![](https://storage.googleapis.com/py_ml_images/second_submission.png)